# <h1 align="center"><font color="red">Extracting text from PDFs and images with LLMs or OCR models</font></h1>

### <font color="gree">Data Scientist.: Dr. Eddy Giusepe Chirinos Isidro</font>

Neste Notebook exploro algumas ferramentas populares de análise de `PDFs` ou Imagens de texto para extração eficiente de informações.
Faço este estudo de várias ferramentas para posteriormente tomar uma decisão e escolher a melhor delas.

Alguns links para mais detalhes:

* [Jaided AI: EasyOCR](https://www.jaided.ai/easyocr/tutorial/)
* [GitHub: EasyOCR](https://github.com/JaidedAI/EasyOCR?tab=readme-ov-file)

In [ ]:
#!pip install -r requirements.txt
#!pip freeze > requirements.txt

# <font color="pink">pypdfium2</font>

In [ ]:
# pip install pypdfium2 matplotlib pillow

import pypdfium2 as pdfium
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO

def convert_pdf_to_images(file_path, scale=300/72):
    
    pdf_file = pdfium.PdfDocument(file_path)  
    page_indices = [i for i in range(len(pdf_file))]
    
    renderer = pdf_file.render(
        pdfium.PdfBitmap.to_pil,
        page_indices = page_indices, 
        scale = scale,
    )
    
    list_final_images = [] 
    
    for i, image in zip(page_indices, renderer):
        
        image_byte_array = BytesIO()
        image.save(image_byte_array, format='jpeg', optimize=True)
        image_byte_array = image_byte_array.getvalue()
        list_final_images.append(dict({i:image_byte_array}))
    
    return list_final_images

In [ ]:
def display_images(list_dict_final_images):
    
    all_images = [list(data.values())[0] for data in list_dict_final_images]

    for index, image_bytes in enumerate(all_images):

        image = Image.open(BytesIO(image_bytes))
        figure = plt.figure(figsize = (image.width / 100, image.height / 100))

        plt.title(f"----- Page Number {index+1} -----")
        plt.imshow(image)
        plt.axis("off")
        plt.show()
        

In [ ]:

convert_pdf_to_images = convert_pdf_to_images('4._Ouvidoria_Geral_1.pdf')

#convert_pdf_to_images

In [ ]:
# Logo convertemos a Imagem o PDF:

#display_images(convert_pdf_to_images)

# <font color="pink">pytesseract</font>

In [ ]:
#!pip install pytesseract -qq

from pytesseract import image_to_string 

def extract_text_with_pytesseract(list_dict_final_images):
    
    image_list = [list(data.values())[0] for data in list_dict_final_images]
    image_content = []
    
    for index, image_bytes in enumerate(image_list):
        
        image = Image.open(BytesIO(image_bytes))
        raw_text = str(image_to_string(image))
        image_content.append(raw_text)
    
    return "\n".join(image_content)

In [ ]:
text_with_pytesseract = extract_text_with_pytesseract(convert_pdf_to_images)

print(text_with_pytesseract)

# <font color="pink">EasyOCR</font>

In [ ]:
#!pip install easyocr

from easyocr import Reader

# Load model for the Portuguese language:
language_reader = Reader(["pt"], gpu=True, recognizer=True, verbose=True, quantize=True)


In [ ]:
# from easyocr import Reader
# help(Reader.__init__)


In [ ]:
def extract_text_with_easyocr(list_dict_final_images):
    
    image_list = [list(data.values())[0] for data in list_dict_final_images]
    image_content = []
    
    for index, image_bytes in enumerate(image_list):
        
        image = Image.open(BytesIO(image_bytes))
        raw_text = language_reader.readtext(image)
        raw_text = "\n".join([res[1] for res in raw_text])
                       
        image_content.append(raw_text)
    
    return "\n".join(image_content)

In [ ]:
text_with_easy_ocr = extract_text_with_easyocr(convert_pdf_to_images)

print(text_with_easy_ocr)

# <font color="gree">Usando `easyocr` diretamente</font>

In [ ]:
import torch
torch.cuda.is_available()


In [ ]:
import easyocr

# Carregar modelo de reconhecimento de texto para o português:
reader = easyocr.Reader(['pt'], gpu=True, recognizer=True, verbose=True, quantize=True)

result = reader.readtext("./data/Atestado_Med_Lucas.jpg", # "./data/Atestado_Med_Lucas.jpg", # "./data/NF_antigo.png", # "./data/RG_internet.jpg"
                         paragraph=True, # Agrupa linhas de texto em parágrafos, facilitando a leitura.
                         x_ths=0.3, # Controlar a sensibilidade da detecção de texto. Valores menores detectam mais texto, mas podem incluir FP.
                         detail=1, # Define o nível de detalhes retornados (1 para caixas de texto, 0 para apenas texto).
                         width_ths=0.9, # Controla a sensibilidade da detecção de texto com base na largura.
                         height_ths=0.2 # Controla a sensibilidade da detecção de texto com base na altura.                     
                        )
#print(result)
text = '\n'.join([line[1] for line in result])
print(text)


# <font color="yellow">Extract Data using `Amazon Textract` API</font>

In [ ]:
#!pip install boto3
import boto3

client = boto3.client('textract',
                      region_name='us-east-1',
                      aws_access_key_id='',
                      aws_secret_access_key='')


# <font color="gree">Tanto o `GPT-4o` quanto o `GPT-4 Turbo` têm capacidades de visão</font>

In [ ]:
import base64

def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
    

In [ ]:
import requests

# Chave Key da API da OpenAI:
api_key = "Micha_chave_key"

# Caminho da sua Imagem:
image_path = "./data/NF_antigo.png"# "./data/Atestado_Med_Lucas.jpg", # "./data/NF_antigo.png", # "./data/RG_internet.jpg"

# Obtendo a string base64:
base64_image = encode_image(image_path)

# Configurar cabeçalhos
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

# Prepare para payload:
payload = {
    "model": "gpt-4o",
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "Extraia informações do seguinte documento."},
                {"type": "image_url", "image_url": {"url":f"data:image/jpeg;base64,{base64_image}"}}
                
            ]
        }
    ],
    "max_tokens": 300
}

In [ ]:
response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

In [ ]:
print(response.json()["choices"][0]["message"]["content"])

<font color="red">Limitações</font>

Embora o `GPT-4 com visão` seja poderoso e possa ser usado em muitas situações, é importante entender as limitações do modelo. Aqui estão algumas das limitações das quais temos conhecimento:

* `Imagens médicas:` O modelo não é adequado para interpretar imagens médicas especializadas, como tomografias computadorizadas, e não deve ser usado para aconselhamento médico.

* `Não inglês:` o modelo pode não ter desempenho ideal ao manipular imagens com texto de alfabetos não latinos, como `japonês` ou `coreano`.

* `Texto pequeno:` Aumente o texto dentro da imagem para melhorar a legibilidade, mas evite cortar detalhes importantes.

* `Rotação:` O modelo pode interpretar incorretamente textos ou imagens girados/de cabeça para baixo.

* `Elementos visuais:` O modelo pode ter dificuldade para entender gráficos ou textos onde cores ou estilos como linhas sólidas, tracejadas ou pontilhadas variam.

* `Raciocínio espacial:` O modelo tem dificuldades com tarefas que exigem localização espacial precisa, como identificar posições de xadrez.

* `Precisão:` O modelo pode gerar descrições ou legendas incorretas em determinados cenários.

* `Formato da imagem:` A modelo tem dificuldades com imagens panorâmicas e olho de peixe.

* `Metadados e redimensionamento:` o modelo não processa nomes de arquivos originais ou metadados, e as imagens são redimensionadas antes da análise, afetando suas dimensões originais.

* `Contagem:` pode fornecer contagens aproximadas de objetos em imagens.

* `CAPTCHAS:` Por motivos de segurança, implementamos um sistema para bloquear o envio de CAPTCHAs.

# <font color="pink">PyPDF2</font>

In [ ]:
#!pip install PyPDF2
from PyPDF2 import PdfReader

def extract_text_with_pyPDF(PDF_File):

    pdf_reader = PdfReader(PDF_File)
    
    raw_text = ''

    for i, page in enumerate(pdf_reader.pages):
        
        text = page.extract_text()
        if text:
            raw_text += text

    return raw_text

In [ ]:
text_with_pyPDF = extract_text_with_pyPDF("4._Ouvidoria_Geral_1.pdf")

print(text_with_pyPDF)

# <font color="pink">Pdfminer</font>

In [ ]:
#!pip install pdfminer.six

from pdfminer.high_level import extract_text

def pdfminer_extract(pdf_path):
  """Extracts text using pdfminer.six."""
  with open(pdf_path, 'rb') as pdf_file:
    text = extract_text(pdf_file)
    return text

In [ ]:
text_with_pdfminer = pdfminer_extract("./4._Ouvidoria_Geral_1.pdf")

print(text_with_pdfminer)

# <font color="pink">pdfplumber</font>

In [ ]:
#!pip install pdfplumber
import pdfplumber

def pdfplumber_extract(pdf_path):
  """Extracts text using pdfplumber."""
  with pdfplumber.open(pdf_path) as pdf:
    text = ''
    for page in pdf.pages:
      text += page.extract_text()
    return text


In [ ]:
text_with_pdfplumber = pdfplumber_extract("./4._Ouvidoria_Geral_1.pdf")

print(text_with_pdfplumber)

# <font color="pink">LangChain</font>

## <font color="gree">Usando `Unstructured`para extrair informações de uma `Imagem`</font>

In [ ]:
# !pip install langchain

from langchain.document_loaders.image import UnstructuredImageLoader

from PIL import Image
from io import BytesIO
import tempfile
import os

# def extract_text_with_langchain_image(list_dict_final_images):
    
#     image_list = [list(data.values())[0] for data in list_dict_final_images]
#     image_content = []
    
#     for index, image_bytes in enumerate(image_list):
        
#         image = Image.open(BytesIO(image_bytes))
#         loader = UnstructuredImageLoader(image)
#         data = loader.load()
#         raw_text = data[index].page_content
                       
#         image_content.append(raw_text)
    
#     return "\n".join(image_content)


def extract_text_with_langchain_image(list_dict_final_images):
    image_list = [list(data.values())[0] for data in list_dict_final_images]
    image_content = []
    
    for index, image_bytes in enumerate(image_list):
        image = Image.open(BytesIO(image_bytes))
        
        # Salva a imagem em um arquivo temporário:
        with tempfile.NamedTemporaryFile(delete=False, suffix='.jpeg') as temp_image_file:
            image.save(temp_image_file, format='JPEG')
            temp_image_path = temp_image_file.name
        
        try:
            loader = UnstructuredImageLoader(temp_image_path)
            data = loader.load()
            raw_text = data[0].page_content  # Index 0 para obter o conteúdo da primeira página
            image_content.append(raw_text)
        finally:
            os.remove(temp_image_path)  # Remove o arquivo temporário
    
    return "\n".join(image_content)
    

In [ ]:
text_with_langchain_image = extract_text_with_langchain_image(convert_pdf_to_images)

print(text_with_langchain_image)

## <font color="blue">Usando `Unstructured`para extrair informações de um `PDF` direto</font>

In [ ]:
import sys
print(sys.getdefaultencoding())

In [ ]:
from langchain.document_loaders import UnstructuredFileLoader

def extract_text_with_langchain_pdf(pdf_file):
    
    loader = UnstructuredFileLoader(pdf_file, encoding='utf-8')
    documents = loader.load()
    pdf_pages_content = '\n'.join(doc.page_content for doc in documents)
    
    return pdf_pages_content
    

In [ ]:
text_with_langchain_files = extract_text_with_langchain_pdf("./data/4._Ouvidoria_Geral_1.pdf")

print(text_with_langchain_files)

In [ ]:
from langchain.document_loaders import UnstructuredFileLoader

loader = UnstructuredFileLoader("./data/4._Ouvidoria_Geral_1.pdf",
                                #mode="elements", # Quando uso este parâmetro traz pouca informação
                                #extract_images_in_pdf=True,
                                #strategy="hi_res",
                                #extract_image_block_output_dir="pasta_eddy"
                                #languages=["por"]
                                #encoding="latin-1" # ou "utf-8"
                               )


docs = loader.load()
pdf_pages_content = '\n\n'.join(doc.page_content for doc in docs)
print(pdf_pages_content)


## <font color="gree">Usando `DirectoryLoader` de `Unstructured`</font>

In [ ]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import UnstructuredFileLoader


DATA_PATH = "./data"

def load_documents():
    loader = DirectoryLoader(DATA_PATH, glob="**/[!.]*", loader_cls=UnstructuredFileLoader, show_progress=True, use_multithreading=True)
    documents = loader.load()
    pdf_pages_content = '\n\n'.join(doc.page_content for doc in documents)
    pdf_pages_content
    return pdf_pages_content
    #return documents


In [ ]:
documents = load_documents()

print(documents)

## <font color="gree">Usando `partition` de `Unstructured`</font>

In [ ]:
# from unstructured.partition.pdf import partition_pdf

# elements = partition_pdf(
#     filename="./data/4._Ouvidoria_Geral_1.pdf",                  # mandatory
#     strategy="fast", #"hi_res",                                     # mandatory to use ``hi_res`` strategy
#     extract_images_in_pdf=False,                            # mandatory to set as ``True``
#     #extract_image_block_types=["Image", "Table"],          # optional
#     extract_image_block_to_payload=False,                  # optional
#     extract_image_block_output_dir="path/to/save/images",  # optional - only works when ``extract_image_block_to_payload=False``
#     #languages=["eng", "swe"]
#     )


# elements

# len(elements)

# import json

# element_dict = [el.to_dict() for el in elements]
# output = json.dumps(element_dict, indent=2)
# print(output)
